In [1]:
from create_tables import *

In [2]:
%run create_tables.py

In [3]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [4]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

# First

In [5]:
# process_data(cur, conn, filepath='data/song_data', func=process_song_file)
# process_data(cur, conn, filepath='data/log_data', func=process_log_file)

In [6]:
filepath='data/song_data'

In [7]:
# get all files matching extension from directory
all_files = []
for root, dirs, files in os.walk(filepath):
    files = glob.glob(os.path.join(root,'*.json'))
    for f in files :
        all_files.append(os.path.abspath(f))

In [8]:
len(all_files)

71

In [9]:
# get total number of files found
num_files = len(all_files)
print('{} files found in {}'.format(num_files, filepath))

71 files found in data/song_data


In [10]:
def process_song_file(cur, filepath):
    # open song file
    df = pd.read_json(filepath, lines=True)

    # insert song record
    song_data = df[["song_id","title","artist_id","year", "duration"]].values[0].tolist()
    cur.execute(song_table_insert, song_data)
    
    # insert artist record
    artist_data = df[["artist_id","artist_name","artist_location","artist_latitude", "artist_longitude"]].values[0].tolist()
    cur.execute(artist_table_insert, artist_data)

In [11]:
func=process_song_file

In [ ]:
# iterate over files and process
for i, datafile in enumerate(all_files, 1):
    func(cur, datafile)
    conn.commit()
    print('{}/{} files processed.'.format(i, num_files))

# Second

In [13]:
def process_log_file(cur, filepath):
    # open log file
    df = pd.read_json(filepath, lines=True)

    # filter by NextSong action
    df = df.query("page == \"NextSong\"")

    # convert timestamp column to datetime
    t = pd.to_datetime(df['ts'], unit='ms')
    df['ts'] = t
    
    # insert time data records
    time_data =  list((t, t.dt.hour, t.dt.day, t.dt.weekofyear, t.dt.month, t.dt.year, t.dt.weekday))
    column_labels =  ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']
    timedict = dict(zip(column_labels,time_data))
    time_df = pd.DataFrame.from_dict(timedict)

    for i, row in time_df.iterrows():
        try:
            cur.execute(time_table_insert, list(row))
        except Exception as ex:
            print(ex, i, row)
            break
        

    # load user table
    user_df = df[["userId","firstName", "lastName", "gender", "level"]]

    # insert user records
    for i, row in user_df.iterrows():
        if row['userId'] != "":
            cur.execute(user_table_insert, row)

    # insert songplay records
    for index, row in df.iterrows():
        
        # get songid and artistid from song and artist tables
        cur.execute(song_select, (row.song, row.artist, row.length))
        results = cur.fetchone()
        
        if results:
            songid, artistid = results
        else:
            songid, artistid = None, None

        # insert songplay record
        songplay_data = (index, row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
        cur.execute(songplay_table_insert, songplay_data)

In [14]:
func=process_log_file

In [15]:
filepath='data/log_data'

In [16]:
# get all files matching extension from directory
all_files = []
for root, dirs, files in os.walk(filepath):
    files = glob.glob(os.path.join(root,'*.json'))
    for f in files :
        all_files.append(os.path.abspath(f))

In [17]:
# get total number of files found
num_files = len(all_files)
print('{} files found in {}'.format(num_files, filepath))

30 files found in data/log_data


In [18]:
all_files[0]

'/home/workspace/data/log_data/2018/11/2018-11-30-events.json'

In [ ]:
# iterate over files and process
for i, datafile in enumerate(all_files, 1):
    func(cur, datafile)
    conn.commit()
    print('{}/{} files processed.'.format(i, num_files))